# MNIST CNN (without BN vs with BN)

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
#%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [3]:
batch_size=64

In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
X_test = np.expand_dims(X_test,1)
X_train = np.expand_dims(X_train,1)

In [6]:
X_train.shape

(60000, 1, 28, 28)

In [7]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [8]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [9]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [10]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [11]:
def norm_input(x): return (x-mean_px)/std_px

## Conv model

In [45]:
def get_conv_model(lr=0.001):
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        ZeroPadding2D((1, 1)),
        Convolution2D(32, 3, 3, activation='relu'),
        MaxPooling2D(),
        ZeroPadding2D((1, 1)),
        Convolution2D(64, 3, 3, activation='relu'),  
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### learning rate 0.001 -> 0.0001

In [88]:
conv_model = get_conv_model(0.001)

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_12 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [89]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_12 (Lambda)               (None, 1, 28, 28)     0           lambda_input_12[0][0]            
____________________________________________________________________________________________________
zeropadding2d_23 (ZeroPadding2D) (None, 1, 30, 30)     0           lambda_12[0][0]                  
____________________________________________________________________________________________________
convolution2d_23 (Convolution2D) (None, 32, 28, 28)    320         zeropadding2d_23[0][0]           
____________________________________________________________________________________________________
maxpooling2d_23 (MaxPooling2D)   (None, 32, 14, 14)    0           convolution2d_23[0][0]           
___________________________________________________________________________________________

In [90]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64) # keras.preprocessing.image.NumpyArrayIterator
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [91]:
conv_model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 15s - loss: 0.1371 - acc: 0.9567 - val_loss: 0.0374 - val_acc: 0.9870


#### Why is loss much higher than val_loss ?

https://faroit.github.io/keras-docs/1.2.2/getting-started/faq/#why-is-the-training-loss-much-higher-than-the-testing-loss

> Besides, the training loss is the average of the losses over each batch of training data. Because your model is changing over time, the loss over the first batches of an epoch is generally higher than over the last batches. On the other hand, the testing loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.

Also, in evaluation, we don't use dropout.

In [92]:
conv_model.optimizer.lr = 0.0001

In [93]:
conv_model.fit_generator(batches, batches.n, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 15s - loss: 0.0506 - acc: 0.9846 - val_loss: 0.0348 - val_acc: 0.9877
Epoch 2/8
60000/60000 [==============================] - 14s - loss: 0.0335 - acc: 0.9895 - val_loss: 0.0262 - val_acc: 0.9912
Epoch 3/8
60000/60000 [==============================] - 15s - loss: 0.0279 - acc: 0.9910 - val_loss: 0.0272 - val_acc: 0.9913
Epoch 4/8
60000/60000 [==============================] - 15s - loss: 0.0224 - acc: 0.9928 - val_loss: 0.0227 - val_acc: 0.9931
Epoch 5/8
60000/60000 [==============================] - 14s - loss: 0.0187 - acc: 0.9938 - val_loss: 0.0280 - val_acc: 0.9911
Epoch 6/8
60000/60000 [==============================] - 14s - loss: 0.0160 - acc: 0.9950 - val_loss: 0.0213 - val_acc: 0.9940
Epoch 7/8
60000/60000 [==============================] - 15s - loss: 0.0133 - acc: 0.9956 - val_loss: 0.0253 - val_acc: 0.9919
Epoch 8/8
60000/60000 [==============================] - 14s - loss: 0.0133 - acc: 0.9957 - val_loss: 0.0205 - 

### learning rate 0.01 -> 0.001

In [94]:
conv_model = get_conv_model(0.01)

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_13 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [95]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64) # keras.preprocessing.image.NumpyArrayIterator
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [96]:
conv_model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 15s - loss: 0.4162 - acc: 0.8715 - val_loss: 0.1998 - val_acc: 0.9390


In [97]:
conv_model.optimizer.lr = 0.001

In [98]:
conv_model.fit_generator(batches, batches.n, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 15s - loss: 0.2737 - acc: 0.9179 - val_loss: 0.1611 - val_acc: 0.9520
Epoch 2/8
60000/60000 [==============================] - 14s - loss: 0.2568 - acc: 0.9219 - val_loss: 0.1607 - val_acc: 0.9503
Epoch 3/8
60000/60000 [==============================] - 14s - loss: 0.2400 - acc: 0.9288 - val_loss: 0.1737 - val_acc: 0.9446
Epoch 4/8
60000/60000 [==============================] - 14s - loss: 0.2329 - acc: 0.9313 - val_loss: 0.1471 - val_acc: 0.9545
Epoch 5/8
60000/60000 [==============================] - 14s - loss: 0.2316 - acc: 0.9309 - val_loss: 0.1436 - val_acc: 0.9574
Epoch 6/8
60000/60000 [==============================] - 14s - loss: 0.2289 - acc: 0.9320 - val_loss: 0.1479 - val_acc: 0.9565
Epoch 7/8
60000/60000 [==============================] - 14s - loss: 0.2278 - acc: 0.9332 - val_loss: 0.1475 - val_acc: 0.9546
Epoch 8/8
60000/60000 [==============================] - 14s - loss: 0.2219 - acc: 0.9352 - val_loss: 0.1436 - 

## Batch normalization

With batch normalization, we can use higher learning rate.

https://deepage.net/deep_learning/2016/10/26/batch_normalization.html

In [99]:
def get_bn_model(lr=0.001):
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        ZeroPadding2D((1, 1)),
        Convolution2D(32, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        ZeroPadding2D((1, 1)),
        Convolution2D(64, 3, 3, activation='relu'),  
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### learning rate 0.001 -> 0.0001

In [100]:
bn_model = get_bn_model(0.001)

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_14 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [101]:
bn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_14 (Lambda)               (None, 1, 28, 28)     0           lambda_input_14[0][0]            
____________________________________________________________________________________________________
zeropadding2d_27 (ZeroPadding2D) (None, 1, 30, 30)     0           lambda_14[0][0]                  
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 32, 28, 28)    320         zeropadding2d_27[0][0]           
____________________________________________________________________________________________________
batchnormalization_20 (BatchNorm (None, 32, 28, 28)    128         convolution2d_27[0][0]           
___________________________________________________________________________________________

In [102]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64) # keras.preprocessing.image.NumpyArrayIterator
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [103]:
bn_model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 16s - loss: 0.1123 - acc: 0.9661 - val_loss: 0.0557 - val_acc: 0.9810


In [104]:
bn_model.optimizer.lr = 0.0001

In [105]:
bn_model.fit_generator(batches, batches.n, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 16s - loss: 0.0511 - acc: 0.9841 - val_loss: 0.0433 - val_acc: 0.9863
Epoch 2/8
60000/60000 [==============================] - 16s - loss: 0.0388 - acc: 0.9876 - val_loss: 0.0339 - val_acc: 0.9876
Epoch 3/8
60000/60000 [==============================] - 16s - loss: 0.0313 - acc: 0.9900 - val_loss: 0.0482 - val_acc: 0.9845
Epoch 4/8
60000/60000 [==============================] - 16s - loss: 0.0291 - acc: 0.9907 - val_loss: 0.0309 - val_acc: 0.9904
Epoch 5/8
60000/60000 [==============================] - 16s - loss: 0.0231 - acc: 0.9922 - val_loss: 0.0269 - val_acc: 0.9917
Epoch 6/8
60000/60000 [==============================] - 16s - loss: 0.0195 - acc: 0.9938 - val_loss: 0.0335 - val_acc: 0.9897
Epoch 7/8
60000/60000 [==============================] - 16s - loss: 0.0193 - acc: 0.9935 - val_loss: 0.0275 - val_acc: 0.9913
Epoch 8/8
60000/60000 [==============================] - 16s - loss: 0.0158 - acc: 0.9948 - val_loss: 0.0369 - 

### learning rate 0.01 -> 0.001

In [106]:
bn_model = get_bn_model(0.01)

/home/tsuyoshi/.pyenv/versions/anaconda3-4.3.0/envs/py2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_15 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [107]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64) # keras.preprocessing.image.NumpyArrayIterator
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [108]:
bn_model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 16s - loss: 0.1534 - acc: 0.9553 - val_loss: 0.1456 - val_acc: 0.9672


In [109]:
bn_model.optimizer.lr = 0.001

In [110]:
bn_model.fit_generator(batches, batches.n, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 16s - loss: 0.0759 - acc: 0.9780 - val_loss: 0.0708 - val_acc: 0.9806
Epoch 2/8
60000/60000 [==============================] - 16s - loss: 0.0652 - acc: 0.9814 - val_loss: 0.1439 - val_acc: 0.9658
Epoch 3/8
60000/60000 [==============================] - 16s - loss: 0.0600 - acc: 0.9836 - val_loss: 0.0886 - val_acc: 0.9807
Epoch 4/8
60000/60000 [==============================] - 16s - loss: 0.0629 - acc: 0.9830 - val_loss: 0.0491 - val_acc: 0.9878
Epoch 5/8
60000/60000 [==============================] - 16s - loss: 0.0509 - acc: 0.9859 - val_loss: 0.0432 - val_acc: 0.9900
Epoch 6/8
60000/60000 [==============================] - 16s - loss: 0.0514 - acc: 0.9869 - val_loss: 0.0599 - val_acc: 0.9868
Epoch 7/8
60000/60000 [==============================] - 16s - loss: 0.0500 - acc: 0.9878 - val_loss: 0.0482 - val_acc: 0.9904
Epoch 8/8
60000/60000 [==============================] - 16s - loss: 0.0507 - acc: 0.9886 - val_loss: 0.0567 - 

| | loss | acc | val_loss | val_acc |
|---|---|---|---|---|
| lr=0.001 -> 0.0001  | 0.0133 | 0.9957 | 0.0205 | 0.9936 | 
| BN, lr=0.001 -> 0.0001 | 0.0158 | 0.9948 | 0.0369 | 0.9889 |
| lr=0.01 -> 0.001 | 0.2219 | 0.9352 | 0.1436 | 0.9582 |
| BN, lr=0.01 -> 0.001 | 0.0507 | 0.9886 | 0.0567 | 0.9877 |
